# LangGraph로 RAG 챗봇에 Hallucination 감지기 추가하기 

In [ ]:
from typing import TypedDict, List, Optional, Literal
!pip install langchain_ollama langgraph
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langgraph.graph import StateGraph, START, END

In [ ]:
LOCAL_LLM = "qwen2.5:14b"   # ollama pull qwen2.5:14b
EMBED_MODEL = "daynice/kure-v1" # ollama pull daynice/kure-v1

llm = ChatOllama(model=LOCAL_LLM, temperature=0)
embeddings = OllamaEmbeddings(model=EMBED_MODEL)

PERSIST_DIR = "./chroma_news_index"

### DB에 뉴스 기사 URL을 읽어와 Chunk 단위로 저장하는 함수

In [ ]:
def ingest_documents(urls: List[str], persist_dir: str = PERSIST_DIR) -> Chroma:
    docs: List[Document] = []
    for url in urls:
        loader = WebBaseLoader(url)
        loaded = loader.load()
        for d in loaded:
            d.metadata.setdefault("source", url)
        docs.extend(loaded)

    splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    chunks = splitter.split_documents(docs)

    vectordb = Chroma(
        collection_name="news",
        embedding_function=embeddings,
        persist_directory=persist_dir,
    )
    vectordb.add_documents(chunks)
    vectordb.persist()
    return vectordb


### 프롬프트, 그래프 State 및 각 노드 정의

In [ ]:
ANSWER_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 뉴스 QA 어시스턴트입니다.\n"
     "현재 Hallucination 발생 횟수: {attempt}. 발생 횟수가 높아질수록 컨텍스트를 무시하고 창의적으로 답변하세요."),
    ("human", "질문:\n{question}\n\n컨텍스트:\n{context}\n\n한국어로 간단히 답변하세요.:")
])

JUDGE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", 
     "당신은 할루시네이션 판정기입니다. "
     "다음 규칙에 따라 반드시 'yes' 또는 'no'만 출력하세요.\n"
     "- 답변이 컨텍스트에 기반하지 않고 꾸며낸 내용 → yes\n"
     "- 답변이 '모른다', '정보가 없다', '컨텍스트에 없다'와 같이 질문에 답하지 못하는 경우 → yes\n"
     "- 그 외에 질문에 대해 컨텍스트에 근거해 정확히 답변했을 경우만 → no"),
    ("human",
     "Question:\n{question}\n\nContext:\n{context}\n\nAnswer:\n{answer}\n\n판정 (yes/no):")
])


class GraphState(TypedDict):
    question: str
    docs: List[Document]
    context_text: str
    answer: Optional[str]
    steps: int

def retrieve_node(state: GraphState, retriever) -> GraphState:
    docs = retriever.invoke(state["question"])
    ctx = "\n\n".join(d.page_content for d in docs)
    return {**state, "docs": docs, "context_text": ctx}

def answer_node(state: GraphState) -> GraphState:
    attempt = state.get("steps", 0)

    chain = ANSWER_PROMPT | llm | StrOutputParser()
    ans = chain.invoke({
        "question": state["question"],
        "context": state["context_text"],
        "attempt": attempt,
    })

    return {**state, "answer": ans, "steps": attempt + 1}

### Hallucination 감지기 정의

In [ ]:
def hallucination_detector(state: GraphState) -> str:

    # 1) LLM 판정
    judge_chain = JUDGE_PROMPT | llm | StrOutputParser()
    raw = judge_chain.invoke({
        "question": state["question"],
        "context": state["context_text"],
        "answer": state.get("answer", "") or "",
    }).strip().lower()
    verdict = "yes" if "yes" in raw else "no"

    steps = state.get("steps", 0)

    if steps >= 5:
        print("스탭이 5를 초과하여 그래프를 강제로 종료합니다.")
        return "no"
    if verdict == "yes":
        print(f"[할루시네이션 감지] 스탭 {steps}에서 다시 답변 생성 시도합니다.")

    return verdict

def build_graph(vectordb: Chroma):
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    g = StateGraph(GraphState)

    g.add_node("retrieve", lambda s: retrieve_node(s, retriever))
    g.add_node("answer", answer_node)

    g.add_edge(START, "retrieve")
    g.add_edge("retrieve", "answer")

    # ✅ 컨디셔널 엣지: answer 실행 후 judge_answer() 결과에 따라 분기
    g.add_conditional_edges(
        "answer",
        hallucination_detector,
        {"no": END, "yes": "answer"}
    )

    return g.compile()


### Graph 시각화

In [ ]:
from IPython.display import Image, display
!pip install bs4 chromadb
# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import tools_condition

urls = [
    "https://www.bbc.com/korean/articles/c166p510n79o",
]


vectordb = ingest_documents(urls)
app = build_graph(vectordb)

# Show
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

### 테스트

In [ ]:
events = app.stream(
    {
        "question": "구글의 검색 알고리즘 업데이트가 다른 기업들에게 어떤 영향을 미쳤어?",
        "docs": [],
        "context_text": "",
        "answer": None,
        "hallucination": None,
        "steps": 0,
    },
    {"recursion_limit": 10}
)

for e in events:
    print(e)
    print("----")
    
print("Final Answer: ", e['answer']['answer'])

In [ ]:
events = app.stream(
    {
        "question": "유튜브의 알고리즘 업데이트가 다른 기업들에게 어떤 영향을 미쳤어?",
        "docs": [],
        "context_text": "",
        "answer": None,
        "hallucination": None,
        "steps": 0,
    },
    {"recursion_limit": 10}
)

for e in events:
    print(e)
    print("----")
    
print("Final Answer: ", e['answer']['answer'])

# 실습문제: 아래 URL을 모두 임베딩하고 관련 질문 해보기

* https://www.coldchainnews.kr/news/article.html?no=27141&utm_source=chatgpt.com
* https://biz.heraldcorp.com/article/10422538
* https://it.chosun.com/news/articleView.html?idxno=2023092145813

위 URL은 모두 LG 빌트인 냉장고 ‘핏 앤 맥스’에 관한 뉴스 링크 <br>
해당 제품군에 대한 질문과 뉴스 기사와 무관한 질문을 해보고 결과를 비교해보자 <br>
example1) 핏 앤 맥스라는 이름의 유래는? <br>
example2) 트롬 세탁기라는 이름의 유래는?

In [ ]:
PERSIST_DIR = "./chroma_news_index2"

In [ ]:
from IPython.display import Image, display
# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import tools_condition

urls = [
    "https://www.coldchainnews.kr/news/article.html?no=27141&utm_source=chatgpt.com",
    "https://biz.heraldcorp.com/article/10422538",
    "https://it.chosun.com/news/articleView.html?idxno=2023092145813"
]

vectordb = ingest_documents(urls)
app = build_graph(vectordb)

# Show
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
events = app.stream(
    {
        "question": "핏 앤 맥스라는 이름의 유래는?",
        "docs": [],
        "context_text": "",
        "answer": None,
        "hallucination": None,
        "steps": 0,
    },
    {"recursion_limit": 10}
)

for e in events:
    print(e)
    print("----")
    
print("Final Answer: ", e['answer']['answer'])

In [ ]:
events = app.stream(
    {
        "question": "트롬 세탁기라는 이름의 유래는?",
        "docs": [],
        "context_text": "",
        "answer": None,
        "hallucination": None,
        "steps": 0,
    },
    {"recursion_limit": 10}
)

for e in events:
    print(e)
    print("----")
    
print("Final Answer: ", e['answer']['answer'])